In [15]:
import pandas as pd
import numpy as np

In [16]:
data = pd.read_csv("../data/Stations.csv")
data.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN


In [17]:
s = data["Daytime Routes"].str.split(expand=True).apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = "Route"
data = data.join(s)
data.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Route
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,N
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,W
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,N
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,W
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,N


In [18]:
data = data[["Line", "Route", "Stop Name", "GTFS Latitude", "GTFS Longitude"]]
data.head()

,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude
0,Astoria,N,Astoria-Ditmars Blvd,40.775036,-73.912034
0,Astoria,W,Astoria-Ditmars Blvd,40.775036,-73.912034
1,Astoria,N,Astoria Blvd,40.770258,-73.917843
1,Astoria,W,Astoria Blvd,40.770258,-73.917843
2,Astoria,N,30 Av,40.766779,-73.921479


In [19]:
groups = data.groupby(["Route", "Line"])
groups.get_group(("N", "Astoria"))

,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude
0,Astoria,N,Astoria-Ditmars Blvd,40.775036,-73.912034
1,Astoria,N,Astoria Blvd,40.770258,-73.917843
2,Astoria,N,30 Av,40.766779,-73.921479
3,Astoria,N,Broadway,40.761820,-73.925508
4,Astoria,N,36 Av,40.756804,-73.929575
5,Astoria,N,39 Av-Dutch Kills,40.752882,-73.932755
6,Astoria,N,Lexington Av/59 St,40.762660,-73.967258
7,Astoria,N,5 Av/59 St,40.764811,-73.973347
462,Astoria,N,Queensboro Plaza,40.750582,-73.940202


In [20]:
routes_within_line = dict(dict())
for (route, line), group_data in groups:
    if not route in routes_within_line.keys():
        routes_within_line[route] = {line : group_data}
    else:
        routes_within_line[route][line] = group_data

In [21]:
for rt, val in routes_within_line.items():
    print(rt)
    print(list(val.keys()), "\n")


1
['Broadway - 7Av'] 

2
['Broadway - 7Av', 'Clark St', 'Eastern Pky', 'Lenox - White Plains Rd', 'Nostrand'] 

3
['Broadway - 7Av', 'Clark St', 'Eastern Pky', 'Lenox - White Plains Rd'] 

4
['Eastern Pky', 'Jerome Av', 'Lexington Av'] 

5
['Dyre Av', 'Eastern Pky', 'Jerome Av', 'Lenox - White Plains Rd', 'Lexington Av', 'Nostrand'] 

6
['Lexington Av', 'Pelham'] 

7
['Flushing'] 

A
['8th Av - Fulton St', 'Liberty Av', 'Rockaway'] 

B
['6th Av - Culver', '8th Av - Fulton St', 'Broadway - Brighton', 'Concourse', 'Queens Blvd'] 

C
['8th Av - Fulton St'] 

D
['4th Av', '6th Av - Culver', '8th Av - Fulton St', 'Concourse', 'Queens Blvd', 'Sea Beach / West End / Culver / Brighton', 'West End'] 

E
['8th Av - Fulton St', 'Queens - Archer', 'Queens Blvd'] 

F
['63rd St', '6th Av - Culver', '8th Av - Fulton St', 'Broadway - Brighton', 'Queens Blvd', 'Sea Beach / West End / Culver / Brighton'] 

G
['6th Av - Culver', '8th Av - Fulton St', 'Crosstown'] 

J
['Jamaica', 'Queens - Archer'] 

L
['

In [22]:
line_orders = {
    '2' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Clark St',
            'Eastern Pky',
            'Nostrand'
            ],
    '3' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Eastern Pky',
            'Clark St'],
    '4' : [ 'Jerome Av',
            'Lexington Av',
            'Eastern Pky' ],
    '5' : [ 'Dyre Av',
            'Lenox - White Plains Rd',
            'Jerome Av',
            'Lexington Av', 
            'Eastern Pky',
            'Nostrand'],
    '6' : [ 'Pelham', 'Lexington Av' ],
    'A' : [ '8th Av - Fulton St',
            'Liberty Av',
            'Rockaway' ],
    'B' : [ 'Concourse',
            '8th Av - Fulton St',
            'Queens Blvd',
            '6th Av - Culver',
            'Broadway - Brighton',
            ], 
    'D' : [ 'Concourse',
            '8th Av - Fulton St', 
            '6th Av - Culver', 
            'Queens Blvd',
            '4th Av', 
            'West End',
            'Sea Beach / West End / Culver / Brighton' ],
    'E' : [ 'Queens - Archer',
            'Queens Blvd',
            '8th Av - Fulton St' ],
    'F' : [ 'Queens Blvd', 
            '63rd St',
            '8th Av - Fulton St',
            '6th Av - Culver',
            'Broadway - Brighton',
            'Sea Beach / West End / Culver / Brighton'],
    'G' : [ 'Crosstown', 
            '8th Av - Fulton St',
            '6th Av - Culver' ],
    'J' : [ 'Queens - Archer', 'Jamaica' ],
    'M' : [ 'Queens Blvd', 
            '6th Av - Culver', 
            'Jamaica', 
            'Myrtle Av' ],
    'N' : [ 'Astoria', 
            'Broadway - Brighton',
            'Manhattan Bridge', 
            '4th Av',
            'Sea Beach', 
           'Sea Beach / West End / Culver / Brighton' ],
    'Q' : [ 'Second Av', 
            '63rd St', 
            'Broadway - Brighton', 
            'Manhattan Bridge', 
            'Sea Beach / West End / Culver / Brighton' ],
    
    'R' : [ 'Queens Blvd',
            'Astoria', 
            'Broadway - Brighton',
            'Broadway',
            '4th Av' ],
    'W' : [ 'Astoria',
            'Broadway - Brighton',
            'Broadway' ],
    'Z' : [ 'Queens - Archer', 'Jamaica' ]
}

In [23]:
full_routes = dict()

single_line_routes = set(routes_within_line.keys()).difference(set(line_orders.keys()))
for rt in single_line_routes:
    key = list(routes_within_line[rt].keys())[0]
    full_routes[rt] = list(routes_within_line[rt][key]["Stop Name"])


for rt, lines in line_orders.items(): 
    full_routes[rt] = []
    for l in lines: 
        full_routes[rt] += list(routes_within_line[rt][l]["Stop Name"])

In [24]:
full_routes.keys()

dict_keys(['L', 'SIR', '7', '1', 'S', 'C', '2', '3', '4', '5', '6', 'A', 'B', 'D', 'E', 'F', 'G', 'J', 'M', 'N', 'Q', 'R', 'W', 'Z'])

In [25]:
# borough hall needs to be moved above nevins st (route 4, 5)
old_loc = full_routes['4'].index("Borough Hall")
full_routes['4'].pop(old_loc)
new_loc = full_routes['4'].index("Nevins St")
full_routes['4'].insert(new_loc,'Borough Hall')

old_loc = full_routes['5'].index("Borough Hall")
full_routes['5'].pop(old_loc)
new_loc = full_routes['5'].index("Nevins St")
full_routes['5'].insert(new_loc, "Borough Hall")

# Rockaway 67 - Far rockaway needs to be swapped with 90th-116th (route A)
loc_90th = full_routes['A'].index("Beach 90 St")
for i in range(4):
    val = full_routes['A'][loc_90th]
    full_routes['A'].remove(val)
    full_routes['A'].append(val)

# 145th needs to be put in between 155th and 135th (route B)
# remove parkside av, beverly rd, cortelyou avenu h, av j, av m (route B)
full_routes['B'].remove("145 St")
full_routes['B'].insert(full_routes['B'].index("135 St"), "145 St")
full_routes['B'].remove("W 4 St-Wash Sq")
full_routes['B'].insert(full_routes['B'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
full_routes['B'].remove("Parkside Av")
full_routes['B'].remove("Beverley Rd")
full_routes['B'].remove("Cortelyou Rd")
full_routes['B'].remove("Avenue H")
full_routes['B'].remove("Avenue J")
full_routes['B'].remove("Avenue M")
full_routes['B'].remove("Avenue U")
full_routes['B'].remove("Neck Rd")

# 145th needs to be put between 155th and 125th (route D)
# 7th ave after 59th st 
# wash square after 34th 
full_routes['D'].remove("145 St")
full_routes['D'].insert(full_routes['D'].index("125 St"), "145 St")
full_routes['D'].remove("7 Av")
full_routes['D'][full_routes['D'].index("W 4 St-Wash Sq")] = "7 Av"
full_routes['D'].insert(full_routes['D'].index('Broadway-Lafayette St'), "W 4 St-Wash Sq")

# ROUTE F
# 'Jay St-MetroTech' before bergen
# 'W 4 St-Wash Sq', before 'Broadway-Lafayette St'
full_routes['F'].remove("Jay St-MetroTech")
full_routes['F'].insert(full_routes['F'].index("Bergen St"), "Jay St-MetroTech")
full_routes['F'].remove("W 4 St-Wash Sq")
full_routes['F'].insert(full_routes['F'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")


# ROUTE M 
#  'W 4 St-Wash Sq', after 14th 
# reverse Myrtle - essex and metropolitan - central
full_routes['M'].remove("W 4 St-Wash Sq")
full_routes['M'].insert(full_routes['M'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
reverse_1 = reversed(full_routes['M'][-13:-7])
reverse_2 = reversed(full_routes['M'][-7:])
full_routes['M'] = full_routes['M'][:-13]
full_routes['M'] += reverse_1
full_routes['M'] += reverse_2

# ROUTE N 
# queenseboro after 39th 
full_routes["N"].remove("Queensboro Plaza")
full_routes["N"].insert(full_routes["N"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE Q
# canal after union square 
full_routes["Q"].remove("Canal St")
full_routes["Q"].insert(full_routes['Q'].index("DeKalb Av"), "Canal St")

# ROUTE R
# dekalb after jay street 
full_routes['R'].remove("DeKalb Av")
full_routes['R'].insert(full_routes['R'].index("Atlantic Av-Barclays Ctr"), "DeKalb Av")

# ROUTE W
# queensboro after 39th
full_routes["W"].remove("Queensboro Plaza")
full_routes["W"].insert(full_routes["W"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE S 
# broken into separate shuttles

In [26]:
stations = data.drop("Line", axis=1)
stations.head()

,Route,Stop Name,GTFS Latitude,GTFS Longitude
0,N,Astoria-Ditmars Blvd,40.775036,-73.912034
0,W,Astoria-Ditmars Blvd,40.775036,-73.912034
1,N,Astoria Blvd,40.770258,-73.917843
1,W,Astoria Blvd,40.770258,-73.917843
2,N,30 Av,40.766779,-73.921479


In [27]:
edges = pd.DataFrame()
edges.reindex(["Route", "To", "From"])
for key, val in full_routes.items():
    cur_route = pd.DataFrame()
    cur_route["To"] = val
    cur_route["From"] = val
    cur_route["Route"] = key
    cur_route["From"] = cur_route["From"].shift(-1)
    cur_route.dropna(inplace=True)    
    edges = edges.append(cur_route)
    
edges = pd.merge(
            edges, stations, left_on=["Route", "To"], right_on=["Route", "Stop Name"]
        ).drop("Stop Name", axis=1
        ).rename({"GTFS Latitude" : "Latitude_1",
                  "GTFS Longitude" : "Longitude_1"},
                axis=1)

edges = pd.merge(
            edges, stations, left_on=["Route", "From"], right_on=["Route", "Stop Name"]
        ).drop("Stop Name", axis=1
        ).rename({"GTFS Latitude" : "Latitude_2",
                  "GTFS Longitude" : "Longitude_2"},
                axis=1)

edges["Distance"] = np.sqrt(
                        (edges["Latitude_1"]-edges["Latitude_2"])**2 +
                        (edges["Longitude_1"]-edges["Longitude_2"])**2
                    )

edges.head()

,To,From,Route,Latitude_1,Longitude_1,Latitude_2,Longitude_2,Distance
0,8 Av,6 Av,L,40.739777,-74.002578,40.737335,-73.996786,0.006286
1,6 Av,14 St-Union Sq.,L,40.737335,-73.996786,40.734789,-73.990730,0.006569
2,14 St-Union Sq.,3 Av,L,40.734789,-73.990730,40.732849,-73.986122,0.005000
3,3 Av,1 Av,L,40.732849,-73.986122,40.730953,-73.981628,0.004878
4,1 Av,Bedford Av,L,40.730953,-73.981628,40.717304,-73.956872,0.028269


In [29]:
edges[["Route", "To", "From", "Distance"]].to_csv("../data/edges.csv")
stations.drop_duplicates("Stop Name").drop("Route", axis=1).to_csv("../data/station_data.csv")